In [1]:
from transformers import TrainingArguments
import evaluate
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer
import numpy as np

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [2]:
from datasets import load_dataset

ds = load_dataset("json", data_files=["/data1/malto/shroom/val.model-agnostic.json"])
ds = ds['train'].train_test_split(train_size=0.8)
ds

DatasetDict({
    train: Dataset({
        features: ['ref', 'labels', 'task', 'hyp', 'tgt', 'src', 'model', 'label', 'p(Hallucination)'],
        num_rows: 399
    })
    test: Dataset({
        features: ['ref', 'labels', 'task', 'hyp', 'tgt', 'src', 'model', 'label', 'p(Hallucination)'],
        num_rows: 100
    })
})

In [3]:
ds['train'][0]

{'ref': 'either',
 'labels': ['Not Hallucination',
  'Not Hallucination',
  'Not Hallucination',
  'Not Hallucination',
  'Hallucination'],
 'task': 'MT',
 'hyp': 'Did you give him my address?',
 'tgt': 'Were you the one who gave him my address?',
 'src': 'Это ты дал ему мой адрес?',
 'model': '',
 'label': 'Not Hallucination',
 'p(Hallucination)': 0.2}

In [4]:
def preprocess_function(examples):
    inputs = [f"Hyp: < {hyp} > Tgt: < {tgt} >" for hyp, tgt in zip(examples["hyp"], examples['tgt'])]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    model_inputs["label"] = [1 if t == "Hallucination" else 0 for t in examples['label']]
    return model_inputs

In [5]:
ds = ds.map(preprocess_function, batched=True)
ds = ds.remove_columns(['hyp', 'ref', 'task', 'p(Hallucination)', 'labels', 'tgt', 'model', 'src'])

Map:   0%|          | 0/399 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [8]:
import evaluate
import numpy as np

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [9]:
id2label = {0: "Not Hallucination", 1: "Hallucination"}
label2id = {"Not Hallucination": 0, "Hallucination": 1}

In [10]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
training_args = TrainingArguments(
    output_dir="local_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=1,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.612800,0.709254,0.510000
2,0.662000,0.673344,0.620000
3,0.604200,0.644670,0.630000
4,0.476300,0.745313,0.670000
5,0.213100,0.677618,0.730000
6,0.165100,0.808624,0.700000
7,0.056000,0.853068,0.710000
8,0.018500,1.085410,0.710000
9,0.011200,1.162407,0.720000
10,0.006600,1.284097,0.720000


TrainOutput(global_step=375, training_loss=0.22052374128003915, metrics={'train_runtime': 23.7564, 'train_samples_per_second': 251.932, 'train_steps_per_second': 15.785, 'total_flos': 88143600085392.0, 'train_loss': 0.22052374128003915, 'epoch': 15.0})